In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

forecast_df = pd.read_csv('data/CHMF_forecast_2019_2021.csv', parse_dates=['V1'])
forecast_df.drop(['Unnamed: 0'], axis=1,inplace=True)
forecast_df.rename(columns={'V1':'Date'}, inplace=True)
forecast_df

In [92]:
forecast_df.set_index('Date', inplace=True)
forecast_df.to_csv('data/CHMF_forecast_2019_2021.csv')
forecast_df.to_excel('data/CHMF_forecast_2019_2021.xls')

In [174]:
filelist = ['data_monthly/ALRS_month_13_21',
            'data_monthly/CHMF_month_13_21',
            'data_monthly/GAZP_month_13_21',
            'data_monthly/GMKN_month_13_21',
            'data_monthly/LKOH_month_13_21',
            'data_monthly/NLMK_month_13_21',
            'data_monthly/NVTK_month_13_21',
            'data_monthly/ROSN_month_13_21',
            'data_monthly/SBER_month_13_21',
            'data_monthly/SNGS_month_13_21',
    
]

In [176]:

for f_name in filelist:
    dateparse = lambda x: pd.datetime.strptime(x, '%b %y')
    openfile = f_name+'.csv'
    mon_data = pd.read_csv(openfile, parse_dates=['Date'], date_parser = dateparse, decimal='.', thousands=',')
    mon_data.drop(['Open','High','Low','Vol.','Change %'], axis=1, inplace=True)
    mon_data.sort_values(by=['Date'], inplace=True, ascending=True)
    mon_data.set_index('Date', inplace=True)
    print(mon_data['Price'])
    mon_data['Log_ret'] = np.log(mon_data['Price']/mon_data['Price'].shift(1))
    mon_data.dropna(inplace=True)
    closefile = f_name+'_clean.csv'
    mon_data.to_csv(closefile)

### Стартовые веса в портфеле

In [324]:
weights_2019 = pd.read_excel('data_weights/jun_weights.xlsx')

In [325]:
weights_2019

,Tickers,Weights
0,ALRS,17.413712
1,CHMF,17.540040
2,GAZP,9.214072
3,GMKN,4.007966
4,LKOH,18.794038
5,NLMK,1.780314
6,NVTK,20.424388
7,ROSN,-1.989332
8,SBER,20.442402
9,SNGS,-7.186032


In [326]:
weights_2019['Weights'] = weights_2019['Weights']/100

In [307]:
weights_2019

,Tickers,Weights
0,ALRS,0.131348
1,CHMF,0.076747
2,GAZP,0.101355
3,GMKN,0.097370
4,LKOH,0.206734
5,NLMK,0.019583
6,NVTK,0.171386
7,ROSN,0.021605
8,SBER,0.224866
9,SNGS,-0.050853


In [179]:
weights_2019.drop(['%'], axis=1, inplace=True)


In [327]:
weights_2019.set_index('Tickers', inplace=True)

In [328]:
weights_2019

,Weights
Tickers,
ALRS,0.174137
CHMF,0.175400
GAZP,0.092141
GMKN,0.040080
LKOH,0.187940
NLMK,0.017803
NVTK,0.204244
ROSN,-0.019893
SBER,0.204424


#### Здесь строится матрица лог-доходностей

In [263]:
return_data_list = [
                    'data_monthly/ALRS_month_13_21_clean.csv',
                    'data_monthly/CHMF_month_13_21_clean.csv',
                    'data_monthly/GAZP_month_13_21_clean.csv',
                    'data_monthly/GMKN_month_13_21_clean.csv',
                    'data_monthly/LKOH_month_13_21_clean.csv',
                    'data_monthly/NLMK_month_13_21_clean.csv',
                    'data_monthly/NVTK_month_13_21_clean.csv',
                    'data_monthly/ROSN_month_13_21_clean.csv',
                    'data_monthly/SBER_month_13_21_clean.csv',
                    'data_monthly/SNGS_month_13_21_clean.csv',
                    ]

In [264]:
alrs = 0
chmf = 0
gazp = 0
gmkn = 0
lkoh = 0
nlmk = 0
nvtk = 0
rosn = 0
sber = 0
sngs = 0
ticker_list = [alrs,chmf,gazp,gmkn,
               lkoh,nlmk,nvtk,
               rosn,sber,sngs,
               ]

In [265]:
def maketidy_rename_and_index(name):
    data = pd.read_csv(name)
    data.rename(columns={"Log_ret":name[13:17]}, inplace=True)
    data.drop(['Price'], axis=1, inplace=True)
    data.set_index('Date', inplace=True)
    return data

In [266]:
for i in range(len(ticker_list)):
    ticker_list[i] = maketidy_rename_and_index(return_data_list[i])

In [267]:
log_ret_matrix = ticker_list[0].merge(ticker_list[1],left_index=True,right_index=True).merge(ticker_list[2],left_index=True,right_index=True).merge(ticker_list[3],left_index=True,right_index=True).merge(ticker_list[4],left_index=True,right_index=True).merge(ticker_list[5],left_index=True,right_index=True).merge(ticker_list[6],left_index=True,right_index=True).merge(ticker_list[7],left_index=True,right_index=True).merge(ticker_list[8],left_index=True,right_index=True).merge(ticker_list[9],left_index=True,right_index=True)

In [329]:
log_ret_matrix_start = log_ret_matrix.loc["2013-01-01":"2019-06-01"]

In [330]:
log_ret_matrix_start

,ALRS,CHMF,GAZP,GMKN,LKOH,NLMK,NVTK,ROSN,SBER,SNGS
Date,,,,,,,,,,
2013-02-01,0.085451,-0.100600,-0.033564,-0.102284,-0.028673,-0.075866,-0.016035,-0.090601,-0.046889,-0.093814
2013-03-01,-0.028942,-0.198458,-0.024460,-0.029102,0.017923,-0.182491,-0.042282,-0.022169,-0.056152,-0.030935
2013-04-01,-0.013991,-0.065373,-0.076946,-0.017101,-0.013095,0.052811,-0.074543,-0.107290,0.002526,-0.039149
2013-05-01,0.029033,0.006468,-0.006059,-0.105145,-0.049094,-0.145678,0.093698,-0.013525,-0.000606,-0.096058
2013-06-01,0.055948,-0.228657,-0.123166,0.023616,0.006581,-0.032052,0.095058,0.074243,-0.055740,0.064654
...,...,...,...,...,...,...,...,...,...,...
2019-02-01,-0.035737,0.029894,-0.023804,0.037392,0.042425,0.041364,-0.058073,-0.030741,-0.047460,-0.062798
2019-03-01,-0.028586,-0.000778,-0.060809,-0.028313,0.069005,0.073359,-0.002038,0.032563,0.031361,-0.032239
2019-04-01,0.019395,0.016593,0.091530,0.044338,-0.067189,0.003513,0.111353,0.039967,0.048919,-0.012541


In [313]:
log_ret_matrix_start.var()

ALRS    0.006561
CHMF    0.008081
GAZP    0.004944
GMKN    0.005320
LKOH    0.002897
NLMK    0.007475
NVTK    0.004587
ROSN    0.005453
SBER    0.008075
SNGS    0.005379
dtype: float64

#### Ковариационная матрица месячных доходностей 2013-2019
$$ \sum $$

In [331]:
cov_matrix_monthly_start = log_ret_matrix_start.cov()

In [332]:
cov_matrix_monthly_start

,ALRS,CHMF,GAZP,GMKN,LKOH,NLMK,NVTK,ROSN,SBER,SNGS
ALRS,0.006493,0.001205,0.000426,0.001514,0.001604,0.001398,0.000368,0.000426,-0.001026,0.000306
CHMF,0.001205,0.007978,0.001758,0.002079,0.001740,0.005107,0.000339,0.001644,0.001063,0.000768
GAZP,0.000426,0.001758,0.004950,0.001260,0.001447,0.001113,0.002277,0.002529,0.003219,0.001490
GMKN,0.001514,0.002079,0.001260,0.005262,0.001857,0.002418,0.001037,0.001354,0.001232,0.002363
LKOH,0.001604,0.001740,0.001447,0.001857,0.002859,0.001691,0.001197,0.002302,0.001805,0.001510
NLMK,0.001398,0.005107,0.001113,0.002418,0.001691,0.007480,0.000305,0.001332,0.000538,0.000950
NVTK,0.000368,0.000339,0.002277,0.001037,0.001197,0.000305,0.004527,0.002263,0.002155,0.001446
ROSN,0.000426,0.001644,0.002529,0.001354,0.002302,0.001332,0.002263,0.005413,0.002612,0.002320
SBER,-0.001026,0.001063,0.003219,0.001232,0.001805,0.000538,0.002155,0.002612,0.007971,0.002457
SNGS,0.000306,0.000768,0.001490,0.002363,0.001510,0.000950,0.001446,0.002320,0.002457,0.005351


In [333]:
cov_matrix_monthly_start.to_excel('data_monthly/cov_matrix_06.xls')

#### $$ \lambda = 2.004 $$

In [334]:
lambda_ = 2.004

In [335]:
PI = lambda_*cov_matrix_monthly_start

In [336]:
PI = PI.dot(weights_2019)

In [337]:
PI

,Weights
ALRS,0.003213
CHMF,0.004952
GAZP,0.004302
GMKN,0.003235
LKOH,0.003645
NLMK,0.003742
NVTK,0.003651
ROSN,0.003663
SBER,0.005097
SNGS,0.002180


In [338]:
PI.to_excel('data_monthly/Pi_06.xls')

In [342]:
log_ret_matrix_test = log_ret_matrix.loc["2019-02-01":"2019-07-01"]

In [343]:
log_ret_matrix_test

,ALRS,CHMF,GAZP,GMKN,LKOH,NLMK,NVTK,ROSN,SBER,SNGS
Date,,,,,,,,,,
2019-02-01,-0.035737,0.029894,-0.023804,0.037392,0.042425,0.041364,-0.058073,-0.030741,-0.047460,-0.062798
2019-03-01,-0.028586,-0.000778,-0.060809,-0.028313,0.069005,0.073359,-0.002038,0.032563,0.031361,-0.032239
2019-04-01,0.019395,0.016593,0.091530,0.044338,-0.067189,0.003513,0.111353,0.039967,0.048919,-0.012541
2019-05-01,-0.066699,-0.008455,0.271542,-0.044483,-0.045000,0.007105,0.093256,0.009747,0.035212,0.015150
2019-06-01,-0.024228,0.029660,0.079206,0.042110,0.006904,-0.076056,0.010826,-0.043665,0.022511,0.054616
2019-07-01,-0.054209,-0.037020,0.017330,0.023348,-0.014908,-0.056680,-0.013399,0.017814,-0.021440,0.026968


In [341]:
log_ret_matrix_test.to_excel('data_monthly/test.xls')

In [348]:
#data_ofz = pd.read_csv('data/OFZ_den_2013_2021.csv')
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
data_ofz = pd.read_csv('data/OFZ_den_2013_2021.csv', parse_dates=['Дата'], date_parser = dateparse, decimal=',')

<ipython-input-348-394dc957fd1c>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')


In [349]:
data_ofz

,Дата,Цена,Откр.,Макс.,Мин.,Изм. %
0,2020-12-30,5.91,5.93,5.96,5.87,"-0,34%"
1,2020-12-29,5.93,5.96,5.99,5.89,"-0,67%"
2,2020-12-28,5.97,5.93,6.02,5.89,"0,67%"
3,2020-12-25,5.93,5.97,5.97,5.90,"-1,00%"
4,2020-12-24,5.99,5.99,5.99,5.94,"0,00%"
...,...,...,...,...,...,...
2115,2013-01-10,6.44,6.75,6.75,6.44,"-2,57%"
2116,2013-01-09,6.61,6.80,6.81,6.61,"-4,34%"
2117,2013-01-08,6.91,6.91,6.91,6.91,"0,88%"
2118,2013-01-03,6.85,6.85,6.85,6.85,"0,00%"


In [350]:
data_ofz.drop(['Откр.', 'Макс.', 'Мин.', 'Изм. %'], axis=1, inplace=True)

In [351]:
data_ofz

,Дата,Цена
0,2020-12-30,5.91
1,2020-12-29,5.93
2,2020-12-28,5.97
3,2020-12-25,5.93
4,2020-12-24,5.99
...,...,...
2115,2013-01-10,6.44
2116,2013-01-09,6.61
2117,2013-01-08,6.91
2118,2013-01-03,6.85


In [352]:
data_ofz.sort_values(by=['Дата'], inplace=True, ascending=True)
data_ofz.set_index('Дата', inplace=True)
data_ofz['Лог_дох'] = np.log(data_ofz['Цена']/data_ofz['Цена'].shift(1))

In [354]:
data_ofz = data_ofz.loc["2019-02-01":"2019-07-01"]

In [357]:
data_ofz.to_excel('data_monthly/ofz.xls')